# Web Scraping Supermarkets Analysis (Group Work)
---

In [2]:
# Packages
import pandas as pd

The first step is to merge the three different CSV files provided by each group member.

In [5]:
# Extract columns of interest for each csv
columns_interest = [
    'ID', 'Competitor', 'Category', 'Product_Description', 'Brand', 
    'Regular_Price (CHF)', 'Grammage', 'Unit', 'Link', 'Scraping_Date', 
    'Discount', 'Actual_Price (CHF)', 'Regular_Price/Unit', 'Actual_Price/Unit']

# Load each CSV file
df1 = pd.read_csv('Final_Scraping_Merged_Pasta.csv', usecols = columns_interest) # Catalina Roth DF
df2 = pd.read_csv('pasta_sauces_merged.csv', usecols = columns_interest) # Martina Diaz DF
df3 = pd.read_csv('RiceData_Cleaned_Transformed_GroupProject.csv', usecols = columns_interest) #Fatima Barcina DF

# Concatenate the DataFrames
df = pd.concat([df1, df2, df3], ignore_index=True)

# Visualize Dtype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   297 non-null    int64  
 1   Competitor           297 non-null    object 
 2   Category             297 non-null    object 
 3   Product_Description  297 non-null    object 
 4   Brand                297 non-null    object 
 5   Regular_Price (CHF)  297 non-null    float64
 6   Grammage             297 non-null    float64
 7   Unit                 297 non-null    object 
 8   Link                 297 non-null    object 
 9   Scraping_Date        297 non-null    object 
 10  Discount             297 non-null    object 
 11  Actual_Price (CHF)   297 non-null    float64
 12  Regular_Price/Unit   297 non-null    float64
 13  Actual_Price/Unit    297 non-null    float64
dtypes: float64(5), int64(1), object(8)
memory usage: 32.6+ KB
